In [1]:
import nltk
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv("training_twitter_x_y_train.csv")
data_test = pd.read_csv("test_twitter_x_test.csv")

In [3]:
data_train.drop(["airline","negativereason_gold","tweet_id","airline_sentiment_gold","name","retweet_count","tweet_coord","tweet_location","user_timezone","tweet_created"],axis=1,inplace=True)
data_test.drop(["airline","negativereason_gold","tweet_id","airline_sentiment_gold","name","retweet_count","tweet_coord","tweet_location","user_timezone","tweet_created"],axis=1,inplace=True)
data_train.columns

Index(['airline_sentiment', 'text'], dtype='object')

In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop+=list(string.punctuation)

In [5]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [6]:
lem=WordNetLemmatizer()

def get_simple_tag(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def NotNumber(w):
    for ch in w:
        if(ch>='0' and ch<='9'):
            return False
    return True
    
def Clean_Words(doc):
    doc=word_tokenize(doc)
    doc = [word for word in doc if word.lower() not in stop and NotNumber(word)==True]
    pos = pos_tag(doc)
    cleaned_words = [lem.lemmatize(w,pos=get_simple_tag(t)).lower() for w,t in pos]
    string=""
    for w in cleaned_words:
        string+=w
        string+=' '
    return string

In [7]:
data_train=[(Clean_Words(doc[1]),doc[0]) for doc in np.array(data_train)]
data_test=[Clean_Words(doc) for doc in np.array(data_test['text'])]

data_train

[('southwestair schedule morning day fact yes .. sure even flight one cancelled flightled ',
  'negative'),
 ('southwestair see worker time time go beyond love fly guy thank ',
  'positive'),
 ('united flew ord miami back great crew service leg thanks ', 'positive'),
 ("southwestair 's horse radish 😤🐴 ", 'negative'),
 ('united flight ord delay air force one last flight sbn mins land ',
  'negative'),
 ('united load u fly sardine knew pilot hour late flight incompetent beyond belief ',
  'negative'),
 ("jetblue stock response delays frustrating poor cust serv amp tell ppl wait amp 'd come back ",
  'negative'),
 ("jetblue 'd nice hoping rack enough mile take trip seattle enjoy perfect latte city coffee ",
  'positive'),
 ('united frankly bad customer service ever problems happen deal defines company never united ',
  'negative'),
 ("southwestair yeah haha never one 's expensive 😂😂 much fun destinationdragons ",
  'positive'),
 ("southwestair mco- gt dca flight almost full people screw m

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec=CountVectorizer(max_features=3000,ngram_range=(1,2),max_df=0.3)
count_vec.fit([doc[0] for doc in data_train])

train_x = count_vec.transform([doc[0] for doc in data_train])
train_y=[doc[1] for doc in data_train]

test_x=count_vec.transform(data_test)

In [9]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.8)
clf.fit(train_x,train_y)

MultinomialNB(alpha=0.8)

In [10]:
test_y=clf.predict(test_x)
np.savetxt('predictions.csv',test_y,fmt='%s')
clf.score(train_x,train_y)

0.816120218579235